## 1. Importar Librerías

In [1]:
import pandas as pd
import numpy as np
import ast
import os


## 2. Cargar los Conjuntos de Datos

In [2]:
# Ruta del archivo CSV
file_path = '../Data/Processed/Clean_data/steam_games_cleaned.csv'

# Leer el archivo CSV en un dataframe
df_games = pd.read_csv(file_path)

# Mostrar las primeras filas del dataframe para verificar que se cargaron correctamente
df_games.head()


,genres,item_name,tags,specs,price,item_id,developer,year
0,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,761140,Kotoshiro,2018
1,"['Free to Play', 'Strategy', 'Indie', 'RPG']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",0.00,643980,Secret Level SRL,2018
2,"['Free to Play', 'Simulation', 'Sports', 'Casu...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult...",0.00,670290,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",['Single-player'],0.99,767400,彼岸领域,2017
4,"['Action', 'Indie', 'Casual', 'Sports']",Log Challenge,"['Action', 'Indie', 'Casual', 'Sports']","['Single-player', 'Full controller support', '...",2.99,773570,NaN,2016


In [3]:
# Ruta del archivo CSV
file_path = '../Data/Processed/Clean_data/user_items_cleaned.csv'

# Leer el archivo CSV en un dataframe
df_items = pd.read_csv(file_path)

# Mostrar las primeras filas del dataframe para verificar que se cargaron correctamente
df_items.head()


,item_id,hours_played,user_id
0,10,0.10,76561197970982479
1,20,0.00,76561197970982479
2,30,0.12,76561197970982479
3,40,0.00,76561197970982479
4,50,0.00,76561197970982479


In [4]:
# Ruta del archivo CSV
file_path = '../Data/Processed/Clean_data/user_reviews_cleaned.csv'

# Leer el archivo CSV en un dataframe
df_reviews = pd.read_csv(file_path)

# Mostrar las primeras filas del dataframe para verificar que se cargaron correctamente
df_reviews.head()


,item_id,recommend,user_id,year,sentiment_analysis
0,10,True,76561198040188061,2011,2
1,10,True,epic_doom,2013,1
2,10,True,mayshowganmore,2014,2
3,10,True,BestinTheWorldThund3r,2014,2
4,10,True,76561198072207162,2014,2


## 3. Crear Conjuntos de Datos para la API

### 3.1. Endpoint 1

Se requiere la creación de un dataset para el primer endpoint de la API, que incluye información sobre desarrolladores de juegos en la plataforma Steam. Se necesita calcular la cantidad de juegos desarrollados por cada desarrollador, así como el porcentaje de juegos gratuitos que ofrecen en relación con su catálogo total. El código utilizado agrupa los datos por desarrollador y año, calcula el conteo de juegos gratuitos y luego determina el porcentaje de juegos gratuitos sobre el total. Además, redondea este porcentaje a dos decimales para mayor claridad.

In [5]:
# Agrupar por desarrollador y contar la cantidad de items desarrollados
developer_counts = df_games.groupby(['developer', 'year'])['item_id'].count().reset_index()
developer_counts.columns = ['developer', 'year', 'item_count']

# Calcular el porcentaje de contenido gratuito para cada desarrollador y año
free_content_counts = df_games[df_games['price'] == 0].groupby(['developer', 'year'])['item_id'].count().reset_index()
free_content_counts.columns = ['developer', 'year', 'free_item_count']

# Combinar los datos de conteo de items y conteo de contenido gratuito por desarrollador
df_ep1 = pd.merge(developer_counts, free_content_counts, on=['developer', 'year'], how='left')

# Calcular el porcentaje de contenido gratuito
df_ep1['free_content_percentage'] = (df_ep1['free_item_count'] / df_ep1['item_count']) * 100
df_ep1['free_content_percentage'] = df_ep1['free_content_percentage'].fillna(0)  # Manejo de valores NaN si no hay contenido gratuito

# Redondear la columna 'free_content_percentage' a 2 decimales
df_ep1['free_content_percentage'] = df_ep1['free_content_percentage'].round(2)

# Eliminar la columna free_item_count del dataframe final
df_ep1.drop(columns=['free_item_count'], inplace=True)

# Imprimir el dataset resultante
df_ep1


,developer,year,item_count,free_content_percentage
0,+7 Software,2016,1,0.0
1,"+Mpact Games, LLC.",2017,1,0.0
2,.M.Y.W.,2016,1,0.0
3,.ez Games,2017,1,0.0
4,07th Expansion,2015,2,0.0
...,...,...,...,...
15042,萌石游戏,2017,1,0.0
15043,高考恋爱委员会,2015,1,100.0
15044,"高考恋爱委员会,Days",2015,1,0.0
15045,"高考恋爱委员会,橘子班",2015,1,0.0


Luego guardamos el Conjunto de Datos para el Endpoint 1

In [6]:
# Define la ruta de destino
output_dir = "../Data/Processed/API_data"
output_file = "ep1.parquet"
output_path = os.path.join(output_dir, output_file)

# Exporta el DataFrame a CSV
df_ep1.to_parquet(output_path, index=False)

### 3.2. Endpoint 2

In [7]:
# Calcular el monto gastado por cada usuario
df_games['price'] = df_games['price'].astype(float)  # Asegurar que el precio sea un tipo numérico
amount_spent = df_items.merge(df_games[['item_id', 'price']], on='item_id', how='left').groupby('user_id')['price'].sum().reset_index()
amount_spent.columns = ['user_id', 'amount']

# Contar la cantidad de items registrados por cada usuario
items_count = df_items.groupby('user_id').size().reset_index(name='items_count')

# Calcular el porcentaje de recomendación para cada usuario
recommendation_count = df_reviews[df_reviews['recommend']].groupby('user_id').size().reset_index(name='recommendation_count')
recommendation_percentage = recommendation_count.merge(items_count, on='user_id', how='left')
recommendation_percentage['recommendation_percentage'] = (recommendation_percentage['recommendation_count'] / recommendation_percentage['items_count']) * 100
recommendation_percentage['recommendation_percentage'] = recommendation_percentage['recommendation_percentage'].round(2)

# Seleccionar solo las columnas relevantes
recommendation_percentage = recommendation_percentage[['user_id', 'recommendation_percentage']]

# Combinar los resultados en un solo dataframe
df_ep2 = amount_spent.merge(recommendation_percentage, on='user_id', how='outer')
df_ep2 = df_ep2.merge(items_count, on='user_id', how='outer')

# Manejar valores NaN si no hay datos para ciertos usuarios
df_ep2 = df_ep2.fillna(0)

# Convertir items_count a entero
df_ep2['items_count'] = df_ep2['items_count'].astype(int)

# Imprimir el dataframe resultante
df_ep2

,user_id,amount,recommendation_percentage,items_count
0,--000--,402.77,0.00,58
1,--ace--,176.80,4.55,44
2,--ionex--,114.91,8.70,23
3,-2SV-vuLB-Kg,442.48,7.35,68
4,-404PageNotFound-,1549.27,0.00,149
...,...,...,...,...
73092,zzonci,19.98,0.00,5
73093,zzoptimuszz,79.95,1.64,61
73094,zzydrax,99.94,0.00,13
73095,zzyfo,828.51,0.00,84


Luego guardamos el Conjunto de Datos para el Endpoint 2

In [8]:
# Define la ruta de destino
output_dir = "../Data/Processed/API_data"
output_file = "ep2.parquet"
output_path = os.path.join(output_dir, output_file)

# Exporta el DataFrame a CSV
df_ep2.to_parquet(output_path, index=False)


### 3.3. Endpoint 3

Primero extraemos la lista con todos los géneros distintos presentes en df_games.genres

In [9]:
# Eliminar filas con valores NaN en la columna 'genres'
df_games.dropna(subset=['genres'], inplace=True)

# Convertir la columna 'genres' a una lista de Python
df_games['genres'] = df_games['genres'].apply(ast.literal_eval)

# Utilizar el método explode para expandir las listas de la columna 'genres'
generos_totales = df_games['genres'].explode()

# Encontrar los géneros únicos
generos_unicos = generos_totales.unique()

# Imprimir la lista de géneros únicos
print("Lista de géneros únicos:")
print(generos_unicos)


Lista de géneros únicos:
['Strategy' 'Action' 'Indie' 'Casual' 'Simulation' 'Free to Play' 'RPG'
 'Sports' 'Adventure' 'Racing' 'Early Access' nan 'Massively Multiplayer'
 'Education' 'Video Production' 'Utilities' 'Web Publishing'
 'Software Training' 'Audio Production' 'Photo Editing' 'Accounting']


In [10]:
# Convertir los géneros únicos en una lista y agregar el valor nulo
generos_unicos_lista = generos_unicos.tolist()

# Lista para almacenar los resultados
resultados = []

# Iterar sobre cada género
for genero in generos_unicos_lista:
    if pd.notnull(genero):  # Verificar que no sea un valor nulo
        # Filtrar los juegos por el género actual
        juegos_por_genero = df_games[df_games['genres'].apply(lambda x: genero in x)]

        # Merge de df_items con juegos_por_genero
        merged_df = df_items.merge(juegos_por_genero, on='item_id')

        # Verificar si merged_df está vacío
        if not merged_df.empty:
            # Encontrar el usuario con más horas jugadas para este género
            usuario_con_mas_horas = merged_df.groupby('user_id')['hours_played'].sum().idxmax()

            # Encontrar las horas jugadas por año para este usuario y género
            horas_por_anio = merged_df[merged_df['user_id'] == usuario_con_mas_horas].groupby('year')['hours_played'].sum()

            # Crear un DataFrame temporal para almacenar los resultados por género
            df_temporal = pd.DataFrame({
                'genre': genero,
                'user_id': usuario_con_mas_horas,
                'year': horas_por_anio.index,
                'hours_year': horas_por_anio.values
            })

            # Agregar el DataFrame temporal a la lista de resultados
            resultados.append(df_temporal)

# Concatenar todos los DataFrames en uno solo
df_ep3 = pd.concat(resultados, ignore_index=True)

# Imprimir el resultado final
df_ep3


,genre,user_id,year,hours_year
0,Strategy,shinomegami,1993,366.68
1,Strategy,shinomegami,1995,341.42
2,Strategy,shinomegami,1997,0.00
3,Strategy,shinomegami,1998,0.00
4,Strategy,shinomegami,1999,0.23
...,...,...,...,...
188,Web Publishing,Xyphien,2012,1077.62
189,Web Publishing,Xyphien,2015,1183.52
190,Software Training,Lickidactyl,2014,1831.93
191,Audio Production,Lickidactyl,2014,1831.93


Luego guardamos el Conjunto de Datos para el Endpoint 3

In [11]:
# Define la ruta de destino
output_dir = "../Data/Processed/API_data"
output_file = "ep3.parquet"
output_path = os.path.join(output_dir, output_file)

# Exporta el DataFrame a CSV
df_ep3.to_parquet(output_path, index=False)


### 3.4. Endpoint 4

In [12]:
# Filtrar las reviews donde 'recommend' es True y 'sentiment_analysis' es 2
reviews_recommended = df_reviews[(df_reviews['recommend'] == True) & (df_reviews['sentiment_analysis'] == 2)]

# Agrupar las reviews por año y contar cuántas hay en cada año
reviews_count_by_year = reviews_recommended.groupby('year').size().reset_index(name='review_count')

# Obtener las desarrolladoras únicas
unique_developers = df_games['developer'].unique()

# Lista para almacenar los DataFrames de resultados por año
resultados_por_año = []

# Iterar sobre cada año
for year in reviews_count_by_year['year']:
    # Filtrar las reviews para el año actual
    reviews_year = reviews_recommended[reviews_recommended['year'] == year]
    
    # Calcular el puntaje de recomendación para cada desarrolladora en este año
    developer_scores = {}
    for developer in unique_developers:
        # Filtrar los juegos de la desarrolladora actual
        games_developer = df_games[df_games['developer'] == developer]
        
        # Contar cuántas reviews positivas tiene la desarrolladora en este año
        reviews_developer = reviews_year[reviews_year['item_id'].isin(games_developer['item_id'])]
        recommend_score = len(reviews_developer)
        
        # Almacenar el puntaje de recomendación de la desarrolladora
        developer_scores[developer] = recommend_score
    
    # Obtener las 3 desarrolladoras con mayor puntaje de recomendación
    top_developers = sorted(developer_scores, key=developer_scores.get, reverse=True)[:3]
    
    # Crear DataFrame para el año actual
    df_resultado = pd.DataFrame(columns=['year', 'developer', 'recommend_score'])
    
    # Agregar las desarrolladoras y sus puntajes al DataFrame de resultados para el año actual
    for developer in top_developers:
        df_resultado = pd.concat([df_resultado, pd.DataFrame({'year': [year], 'developer': [developer], 'recommend_score': [developer_scores[developer]]})], ignore_index=True)
    
    # Agregar el DataFrame del año actual a la lista de resultados por año
    resultados_por_año.append(df_resultado)

# Concatenar todos los DataFrames de resultados por año en uno solo
df_ep4 = pd.concat(resultados_por_año, ignore_index=True)

# Imprimir el DataFrame de resultados
df_ep4

,year,developer,recommend_score
0,2010,Valve,16
1,2010,Tripwire Interactive,3
2,2010,Sven Co-op Team,3
3,2011,Valve,107
4,2011,Tripwire Interactive,15
5,2011,Re-Logic,14
6,2012,Valve,339
7,2012,Re-Logic,33
8,2012,Facepunch Studios,25
9,2013,Valve,846


Luego guardamos el Conjunto de Datos para el Endpoint 4

In [13]:
# Define la ruta de destino
output_dir = "../Data/Processed/API_data"
output_file = "ep4.parquet"
output_path = os.path.join(output_dir, output_file)

# Exporta el DataFrame a CSV
df_ep4.to_parquet(output_path, index=False)

### 3.5. Endpoint 5

In [14]:
df_temporal = df_games[['developer', 'item_id']].sort_values(by='developer')
df_temporal

,developer,item_id
21261,+7 Software,459830
15086,"+Mpact Games, LLC.",349510
20548,.M.Y.W.,553830
16848,.ez Games,647050
26696,07th Expansion,310360
...,...,...
32116,NaN,775640
32117,NaN,777930
32118,NaN,775370
32119,NaN,777950


In [15]:
merged_df = pd.merge(df_temporal, df_reviews, on='item_id', how='left')
merged_df

,developer,item_id,recommend,user_id,year,sentiment_analysis
0,+7 Software,459830,NaN,NaN,NaN,NaN
1,"+Mpact Games, LLC.",349510,NaN,NaN,NaN,NaN
2,.M.Y.W.,553830,NaN,NaN,NaN,NaN
3,.ez Games,647050,NaN,NaN,NaN,NaN
4,07th Expansion,310360,True,sanaret,2015.0,0.0
...,...,...,...,...,...,...
70065,NaN,775640,NaN,NaN,NaN,NaN
70066,NaN,777930,NaN,NaN,NaN,NaN
70067,NaN,775370,NaN,NaN,NaN,NaN
70068,NaN,777950,NaN,NaN,NaN,NaN


In [16]:
merged_df.drop(['recommend', 'user_id', 'year'], axis=1, inplace=True)
merged_df

,developer,item_id,sentiment_analysis
0,+7 Software,459830,NaN
1,"+Mpact Games, LLC.",349510,NaN
2,.M.Y.W.,553830,NaN
3,.ez Games,647050,NaN
4,07th Expansion,310360,0.0
...,...,...,...
70065,NaN,775640,NaN
70066,NaN,777930,NaN
70067,NaN,775370,NaN
70068,NaN,777950,NaN


In [17]:
# Llenar los valores NaN en la columna 'sentiment_analysis' con 1 (neutro)
merged_df['sentiment_analysis'].fillna(1, inplace=True)

# Convertir la columna 'sentiment_analysis' a enteros
merged_df['sentiment_analysis'] = merged_df['sentiment_analysis'].astype(int)

# Contar las reviews positivas y negativas y crear las columnas correspondientes
merged_df['positive_reviews'] = (merged_df['sentiment_analysis'] == 2).astype(int)
merged_df['negative_reviews'] = (merged_df['sentiment_analysis'] == 0).astype(int)

# Eliminar la columna 'sentiment_analysis'
merged_df.drop('sentiment_analysis', axis=1, inplace=True)

# Agrupar por desarrollador y sumar las reviews positivas y negativas
developer_reviews = merged_df.groupby('developer').agg({
    'positive_reviews': 'sum',
    'negative_reviews': 'sum'
}).reset_index()
merged_df


C:\Users\User\AppData\Local\Temp\ipykernel_24556\1271713405.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['sentiment_analysis'].fillna(1, inplace=True)


,developer,item_id,positive_reviews,negative_reviews
0,+7 Software,459830,0,0
1,"+Mpact Games, LLC.",349510,0,0
2,.M.Y.W.,553830,0,0
3,.ez Games,647050,0,0
4,07th Expansion,310360,0,1
...,...,...,...,...
70065,NaN,775640,0,0
70066,NaN,777930,0,0
70067,NaN,775370,0,0
70068,NaN,777950,0,0


In [18]:
# Eliminar la columna 'item_id'
merged_df.drop('item_id', axis=1, inplace=True)

# Agrupar por desarrollador y sumar las reviews positivas y negativas
developer_reviews = merged_df.groupby('developer').agg({
    'positive_reviews': 'sum',
    'negative_reviews': 'sum'
}).reset_index()
merged_df

,developer,positive_reviews,negative_reviews
0,+7 Software,0,0
1,"+Mpact Games, LLC.",0,0
2,.M.Y.W.,0,0
3,.ez Games,0,0
4,07th Expansion,0,1
...,...,...,...
70065,NaN,0,0
70066,NaN,0,0
70067,NaN,0,0
70068,NaN,0,0


In [19]:
# Obtener los desarrolladores únicos
unique_developers = merged_df['developer'].unique()

# Lista para almacenar los DataFrames por desarrollador
developer_dfs = []

# Iterar sobre los desarrolladores únicos
for developer in unique_developers:
    # Filtrar las filas correspondientes al desarrollador actual
    developer_rows = merged_df[merged_df['developer'] == developer]
    
    # Sumar las revisiones positivas y negativas para el desarrollador actual
    positive_reviews_sum = developer_rows['positive_reviews'].sum()
    negative_reviews_sum = developer_rows['negative_reviews'].sum()
    
    # Crear un DataFrame para el desarrollador actual
    developer_df = pd.DataFrame({
        'developer': [developer],
        'positive_reviews': [positive_reviews_sum],
        'negative_reviews': [negative_reviews_sum]
    })
    
    # Agregar el DataFrame del desarrollador a la lista
    developer_dfs.append(developer_df)

# Concatenar todos los DataFrames de desarrolladores en uno solo
df_ep5 = pd.concat(developer_dfs, ignore_index=True)

df_ep5.head()

,developer,positive_reviews,negative_reviews
0,+7 Software,0,0
1,"+Mpact Games, LLC.",0,0
2,.M.Y.W.,0,0
3,.ez Games,0,0
4,07th Expansion,0,1


In [20]:
# Define la ruta de destino
output_dir = "../Data/Processed/API_data"
output_file = "ep5.parquet"
output_path = os.path.join(output_dir, output_file)

# Exporta el DataFrame a Parquet
df_ep5.to_parquet(output_path, index=False)